# Assignment 7: Databases and SQL

Please read the tasks description carefully and implement **only** what the tasks ask you to implement. Closely following the task descriptions will be beneficial, so keep your divergence in check&mdash;the test cases below each input cell are the gold standard. Finally, for this assignment, you do not need any error handling, you can assume that all input to your function will be valid.

As for the other assignments, using `print` is encouraged to test your implementation but is never required. Make sure not to confuse `return` and `print` statements: If your function has to **return** something, use the `return` statement. 

Try to implement the tasks yourself or in a small team. If you blindly copy a solution from the Internet or other students, you will not take home any learnings. Rather, make an effort to understand the solution! Furthermore, do not modify the _test cells_&mdash;if you do, you effectively cheat the system which is not helpful for your learning process.

Some aspects of this assignment require you to <strong>self-study</strong> and do some research beyond the lecture contents&mdash;use your favorite search engine to look up documentation, usage examples, and definitions of the mentioned functions. There might be tasks where you have to read and investigate the [Python Standard Library](https://docs.python.org/3/library/) to find the documentation for a function that is used or that you want to use.

This assignment will use the third-party module [pandas](https://pandas.pydata.org/). It will also use [sqlite3](https://docs.python.org/3/library/sqlite3.html) for accessing the database file.

In Google Colab and Anaconda, it is already installed. If you see an `ImportError` in the next cell, run `%pip install pandas` to install this module.

<p style="padding: 1em; background-color: #ffa62f;">We'll load the file <tt>database.sqlite</tt> throughout the assignment. <strong> Make sure that it is in the <i style='color: #30AA30'>same</i> path as the notebook. When running in Google Colab, you have to upload this file everytime you open the assignment.</strong></p>

<p style="padding: 1em; background-color: #ffa62f;">If you run into an error with inserting, deleting, or otherwise modifying the datase, the easiest solution is to <strong>Restart the Kernel/Runtime</strong> and <strong>Reupload/Overwrite</strong> the database file with the original one.</p>

---
# Task 0: Loading the database file.

For **reading** from a database, we'll use [Pandas](https://pandas.pydata.org/), for **writing** we'll use the included `sqlite3` module.

In [ ]:
import sqlite3
import pandas as pd

Execute the next cell to try to load the database file. You'll see an error if it doesn't load correctly.


Afterwards, there will be the variable `DB` which holds the **Connection** to the database.

In [ ]:
from pathlib import Path

DB_FILE = Path('database.sqlite')
if not DB_FILE.exists():
    print("\033[1;41m", " " * 45, '\n', "      Please upload the the Database file.    \n", " " * 45, "\033[0m")
else:
    print("\033[1;42m", " " * 45, '\n', "The database file was found and can be loaded \n", " " * 45, "\033[0m")

In [ ]:
DB = sqlite3.connect(DB_FILE)
with DB:
    DB.execute("PRAGMA foreign_keys = 1") # enforce foreign keys, see below

## Database Structure

Here, we have an overengineered database structure with more tables than we need. However, this benefits exercising massive joins over multiple tables.
We have students, who have names and a date of birth. Each student is registered in our university and has therefore a `student_id` and a starting semester (`immatriculation`). If the `exmatriculation` semester is not set (meaning, it is `NULL`), this student is currently actively registered.
Each student is enrolled into 1 or 2 courses per semester and will have a grade for this enrollment. This university allows students to enroll into the same course multiple times and also take the exam every time.

The ID for each table is always called `id`. So, if you use `pd.read_sql`, you can always tell _pandas_ to use `index_col=['id']`.

The database has the following tables. The database has primary keys AND foreign keys. They are enforced, meaning you will receive an error if you add an entry with an ID that already exists or you want to add an entry with a foreign key that does not exist in the reference table.


The following constraints are given:
   - `INT`: this value is an integer
   - `REAL/FLOAT`: this value is a float
   - `STRING`: this value a string
   - `PRIMARY KEY`: this is a unique identifier for this table
   - `FOREIGN KEY`: this is an identifier for the mentioned table
   - `NOT NULL`: this value can't be NULL/None
   - `NULLABLE`: this value *can* be NULL/None

- Table `students`:  
    - id: INT, PRIMARY KEY, NOT NULL
    - first_name: STRING, NOT NULL
    - last_name: STRING, NOT NULL
    - date_of_birth: DATE (YYYY-MM-DD), NOT NULL
- Table `registrations`:
    - id: INT, PRIMARY KEY, NOT NULL
    - student_id: INT, FOREIGN KEY (students.id), NOT NULL
    - immatriculation: STRING (e.g. "2021FS"), NOT NULL
    - exmatriculation: STRING (e.g. "2021FS"), **NULLABLE**
- Table `courses`:
    - id: INT, PRIMARY KEY, NOT NULL
    - name: STRING, NOT NULL
    - category: STRING, NOT NULL
- Table `enrollments`:
    - id: INT, PRIMARY KEY, NOT NULL
    - student_id: INT, FOREIGN KEY (student.id), NOT NULL
    - course_id: INT, FOREIGN KEY (courses.id), NOT NULL
    - semester: STRING (e.g. "2021FS"), NOT NULL
- Table `grades`:
    - id: INT, PRIMARY KEY, NOT NULL
    - enrollment_id: INT, FOREIGN KEY (enrollment.id), NOT NULL
    - grade: REAL/FLOAT, NOT NULL (Value between 1.0-6.0 in 0.25 steps)

In [ ]:
# Example Entries for each table:
print('Table "students"\n',      pd.read_sql("""SELECT * FROM students LIMIT 5""",      DB, index_col=['id']), '\n')
print('Table "registrations"\n', pd.read_sql("""SELECT * FROM registrations LIMIT 5""", DB, index_col=['id']), '\n')
print('Table "courses"\n',       pd.read_sql("""SELECT * FROM courses LIMIT 5""",       DB, index_col=['id']), '\n')
print('Table "enrollments"\n',   pd.read_sql("""SELECT * FROM enrollments LIMIT 5""",   DB, index_col=['id']), '\n')
print('Table "grades"\n',        pd.read_sql("""SELECT * FROM grades LIMIT 5""",        DB, index_col=['id']), '\n')

<p style="background:rgba(255, 232, 47, 0.4)">
Please note that the assignment uses the <b>German Notation of the Semester</b>. So, the "Fall Semester 2021" would be "2021HS", the "Spring Semester 2021" will be "2021FS". This will be important for at least Task 4.2 and Task 6.<br>
Due to practical reasons the most current semester used in this assignment is "2021HS".</p>

---
# Task 1: Playing with students

Use the given database `DB` to query the subsequent tasks.

Each task can be solved with one single query. Try **not** to use Python or Pandas methods for sorting, selecting, or other purposes.

However, you can use multiple queries and use the intermediate results for the next query.

## Task 1.1: How Many Students?

Query the database `DB` and count how many students are in the database. Use the `pd.read_sql()` function with your query as the first parameter and the database connection `DB` as the second parameter.
We don't care about the ID, and using an ID with an aggregation function like `COUNT()`, doesn't return one anyway, so do **not** tell pandas that there is an `index_col`.

Save the result of the query in the variable `number_of_students`.

<p style="background-color: #ffa62f; padding: 1em">
Later on, Task 2 wants you to to <strong>modify</strong> the database.<br />
If you already executed the solution of Task 2, this will result in the test cases in this Task 1 to fail! <br/><br/>
The total number of students in the database is <strong>4269</strong>, however, if you add one
student in Task 2.1, of course, then the new total number is <strong>4270</strong>. <br /><br/>
This will stop a previously succeeding test case. Do not worry. Try to understand the error message!
</p>

In [ ]:
# Add your code here:



In [ ]:
# Test Case
from unittest import TestCase
test_case = TestCase()

# Sanity
test_case.assertTrue('number_of_students' in locals(), msg='You have to call the result `number_of_students`.')
test_case.assertIsInstance(number_of_students, pd.DataFrame, msg='Store the result directly in the variable.')

# Actual Test
test_case.assertEqual(1, len(number_of_students), msg="The result can only have one result.")
test_case.assertEqual(4269, number_of_students.iloc[0].item(), msg="The number in your result is not correct.")


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

## Task 1.2: Students that share a specific birthday

Query the database `DB` and return the names (both `first_name` and `last_name`) who have their birthday on **Oct 7th, 2000**.

Store the result of the query using `pd.read_sql` in the variable `oct_7th`.

_Hint: In SQL, you can generate a new date with the `DATE()` function. It takes one SQL string is the format YYYY-MM-DD and creates an internal date object, which is compatible with the `date_of_birth` column of the `students` table._

For printing the result, or accessing the results, you can use the `.itertuples()` method of a query result. What you get back is an _iterable_ that contains the result from your query as tuples. If you gave _pandas_ the parameter `index_col`, the first entry in the tuple will be the actual `id` of this row. The next items in the tuple are in the same order as you gave in the `SELECT` statement.

For this task, you are required to only select the id, first, and last name from the students. So, when calling `oct_7th.itertuples()`, you'll receive a tuple with three entries: (index, first_name, last_name).

In [ ]:
# Add your code here:



In [ ]:
# Print the results:
print("Following people share their birthday on Oct 7th, 2000")
for i, first_name, last_name in oct_7th.itertuples():
    print(f' - {first_name} {last_name}')

In [ ]:
# Test Case
from unittest import TestCase
test_case = TestCase()

# Sanity
test_case.assertTrue('oct_7th' in locals(), msg='You have to call the result `oct_7th`.')
test_case.assertIsInstance(oct_7th, pd.DataFrame, msg='Store the result directly in the variable.')

# Actual Test
test_case.assertEqual(3, len(oct_7th), msg="The result must have four entries.")
test_case.assertEqual(oct_7th.columns.to_list(), ['first_name', 'last_name'], msg='You have to select only the first and last name.')

# Equality
test_case.assertEqual(
    {('Kenda Suellen', 'Ficenec'), ('Quinn Patricia', 'Grisom'), ('Audria', 'Gronquist')},
    set((_i.first_name, _i.last_name) for _i in oct_7th.itertuples()))


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

---
## Task 1.3: Students who are 25 years old.

Query the `DB` for students' first and last name who are 25 years old. Save the result of the query in the variable `age_25`. If you select the `id` as well, you have to tell pandas that this column is the index column.

As the reference date you **must** use December 1st, 2021. You can generate this date with `DATE('2021-12-01')`. If you fail to use this date, the test will fail. (In a real-live scenario, you would, of course, use the current day.)

_Hint: You can compute a student's age by the difference from the current day and the date of birth. Example:_
```sql
SELECT DATE('2010-01-01') - DATE('2000-01-01')
```
⇒ 10 (years)

Do not overcomplicate the age computation. Use above computation as template.

In [ ]:
# Add your code here:



In [ ]:
# Test Case
from unittest import TestCase
test_case = TestCase()

# Sanity Check
test_case.assertTrue('age_25' in locals(), msg='You have to call it `age_25`.')
test_case.assertIsInstance(age_25, pd.DataFrame, msg='Store the result directly in `age_25`.')

# Check of length
test_case.assertEqual(age_25.columns.to_list(), ['first_name', 'last_name'], msg='You have to select only the first and last name.')
test_case.assertEqual(len(age_25), 699, msg='The resulting list MUST have 679 entries. It seems like your WHERE clause is not correct.')


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

---
## Task 1.4: Student ID

Query the database for the Student ID of the student _Luanna Guzzo_. Use pandas' index column feature.

Save the result of the query directly as the variable `luanna_id`.

In [ ]:
# Add your code here:



In [ ]:
# For future reference, we convert this value to its actual integer:
actual_luanna_id = int(luanna_id.index[0])

In [ ]:
# Test Case
from unittest import TestCase
test_case = TestCase()

# Sanity
test_case.assertTrue('luanna_id' in locals(), msg='You have to call the result `luanna_id`.')
test_case.assertIsInstance(luanna_id, pd.DataFrame, msg='Store the result directly in the variable.')

# Actual Test
test_case.assertEqual(1, len(luanna_id), msg="The result must have one entry.")
test_case.assertEqual(luanna_id.columns.to_list(), [], msg='There will be no columns because the selected entry is the ID.')

# Equality
test_case.assertEqual(50952, luanna_id.index)


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

---
# Task 2: Modifying the Database
Be careful when using `INSERT`, `DELETE`, and `UPDATE` statements because they **modify the database**. If you find yourself in trouble, you can always replace the database file `database.sqlite` **AND** reload the database (see Task 0) or restart the kernel.

Because of _foreign key_ constraints, you have to be careful with the order of inserting.

When writing into the database, you **HAVE TO** use the following structure

```python
with DB:
    DB.execute("""INSERT INTO ...""")
```

If you don't do this, you might end up with a **locked** database. A kernel restart is your only solution then.

After modifying the database, the test cases for Task 1 might fail, especially the ones for counting the students because you're adding a new student in Task 2.1. So make sure, that your code works **BEFORE** modifying the database.

---
## Task 2.1: Adding yourself to the Database.
You want to add yourself to the database and enroll into a few courses and receive grades. However, there is no nice interface, so you have to work on the real database.
None of the ids are auto-generated, so you have to always use an ID that is not in use. The description below tells you which one you have to set.


Add yourself to the database:

1. Add yourself into the `students` table
    - Use your actual first and last name.
    - The date_of_birth must be of form "YYYY-MM-DD" (e.g., "1994-03-14")
    - As student id, use the integer 60000.
1. Register with the university
    - Add a new entry to the `registration` table.
    - Use the registration id 10000.
    - Use the last semester "2021HS" as immatriculation.
    - Either set exmatriculation to NULL or don't set it at all (it defaults to NULL)
1. Enroll into at least 2 courses (You probably have to find all courses first)
    - Use a SELECT to show all courses and print them using `.itertuples()` as demonstrated in Task 1.2.
    - Choose 2 courses and INSERT them into the enrollments table.
    - Start with ID 60000 for adding your enrollments.
    - Use "2021HS" as semester.
1. Add grades
    - Add grades of your liking for you for the courses you just added.
    - The grade id will be the same as the enrollment ids (starting from 60000 and upwards)
    


There is no test case for this task.

Use `SELECT`s in different cells to verify you added the entries correctly.

In [ ]:
# Add your code here:



---
## Task 2.2: Updating Categories

If you look at the course categories more closely, you can see that the 3 courses with the highest ID are in the wrong category _medicine_. They should be in the category _computer science_.

Use an `UPDATE` statement to change the category for (**and only for**) these 3 entries.

In [ ]:
# Add your code here:



---
# Task 3: Courses of one student

Use the just found id of **Luanna Guzzo** (`actual_luanna_id`) to find all the names of all courses that she is and was enrolled in any semester. Use a `SELECT DISTINCT` to remove duplicate enrollments.

You don't need to use a variable but simply copy the actual ID into your query of the result from Task 1.4. (If you didn't find it, use the ID `50952`.)

Store your result in the variable `course_names`.

Unfortunately, the course name is not stored in the `enrollments` table. So you can either query **multiple** tables at the same time (`JOIN`) or first extract the appropriate distinct ids and then hardcode them into a new query.

_Hint:_ Similar to Python, you can check if a value is in a given list in the `WHERE` clause:

```sql
...
WHERE id IN (1, 2, 3, 4, 5)
...
```


In [ ]:
# Add your code here:



In [ ]:
# Test Case
from unittest import TestCase
__ = TestCase()

# Sanity Check
__.assertTrue('course_names' in locals(), msg='You have to call it `course_names`.')
__.assertIsInstance(course_names, pd.DataFrame, msg='Store the result directly in `course_names`.')

# Check of length
__.assertEqual(len(course_names.columns), 1, msg='You have to select only the name of the course (The result has ONE column.)')
__.assertEqual(len(course_names), 9, msg='The resulting list must have 6 entries. Did you use DISTINCT?')

# Actual Check
__.assertEqual(
    {'A Complex Adaptive Systems perspective on Swarm Intelligence',
     'Advanced Applications of Gene-Splicing',
     'Beginner Micro-Biology of Implants',
     'Differences in Jupiter and Luna Law',
     'Fundamentals of Open Systems',
     'Fundamentals of Space-Time-Travel',
     'Historic Reasons for Using the Swiss Grading System',
     'Introduction to 5G-Brainwave Interception',
     'Musical Co-Creation with Clones'},
    set(course_names.iloc[:, 0])
)


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

---
# Task 4: Average Grade for one specific Student

Now, you know the ID and enrolled courses of Mrs. Guzzo. The next step is to compute the average of all her grades.

For this task, you should use **all** grades from this student, although she took multiple courses multiple times.

Store the result of the query in `average_grade`.

Instead of a join, which you can use, of course (and is encouraged), you can use multiple queries to find the enrollments and grades for this one student


_Hint: Computing the average is done by the function `AVG()` (analog to `COUNT()`) in the `SELECT` clause._

In [ ]:
# Add your code here:



In [ ]:
# Test Case
from unittest import TestCase
test_case = TestCase()

# Sanity Check
test_case.assertTrue('average_grade' in locals(), msg='You have to call it `rabb_average_grade`.')
test_case.assertIsInstance(average_grade, pd.DataFrame, msg='Store the result directly in `rabb_average_grade`.')

# Check of length
test_case.assertEqual(len(average_grade.columns), 1, msg='You have to select only the average of the grades (The result has ONE column.)')
test_case.assertEqual(len(average_grade), 1, msg='The resulting list must have 1 entry.')

# Actual Check
test_case.assertEqual(3.73, average_grade.round(2).iloc[0].item(), msg='The average grade should be 3.5 (if rounded to 2 decimals).')


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

---
## Task 4.2: Top 10 Students

You want to find new tutors for next semester, so you need good students. For this, you want the 10 best students that are currently registered (meaning that they are not exmatriculated) and that are registered for at least 3 semesters, meaning their immatriculation semester is at least "2021FS" or earlier.

Select the id, first name, last name, and average of the 10 best students. Use `index_col=['id']` to tell pandas that the `id` column is the one for the index.

Save the result of the query in the variable `top10`.

The students are ranked by their average grades of all their courses, multiple enrollments count also towards this average.

You will need all tables except `courses`. 

For the average of all students, you need the `GROUP BY` clause. There might be students with the same name combiation, so you have to find a better way for **id**entifying a student.

If you have the averages of all students, how can you select the ones with the highest average? How would you do it Python? The `ORDER BY` clause will help you.

The clause `LIMIT X` limits the number of returned rows to `X`, so set this wisely.

In [ ]:
# Add your code here:



In [ ]:
# Test Case
from unittest import TestCase
test_case = TestCase()

# Sanity Check
test_case.assertTrue('top10' in locals(), msg='You have to call it `top10`.')
test_case.assertIsInstance(top10, pd.DataFrame, msg='Store the result directly in `top10`.')

# Check of length
test_case.assertEqual(len(top10.columns), 3, msg='You have to select the first name, last name, and average.')
test_case.assertEqual(len(top10), 10, msg='The resulting list must have 10 entries.')

# Actual Check
test_case.assertEqual(set(top10.index), set([51623, 52504, 50020, 52076, 53627, 52928, 54258, 51132, 52343, 52645]), msg='Your list differs.')


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

---
# Task 5: Average grade for any student

Now, we want to define a Python function `compute_average_grade(first_name, last_name)` that returns the average grade of the given student. (Like above, you have to return the result of the query.)

For this task, you can assume that there are no two students with the same first and last name.

You may want to test this by calling your function with the student _Luanna Guzzo_ in order to compare results with Task 4.

To set the parameters (the first and last name) **do not use format strings** to insert the values into the query. Instead use a `?` (question mark) where the parameter should be placed and use the `params=()`  paramter in the `pd.read_sql()` method to set the tuple in the order the questions marks appear.

Example:
```python
pd.read_sql(
    "SELECT * FROM students WHERE id = ?", # <-- one question mark where the ID should be
    DB,
    index_col=['id'],
    params=(60000,)  # <-- Must be a Tuple, even if only one value!
)
```
Multiple question marks correspond to multiple entries in the `params=()` tuple. The order is the ones the question marks appear!

_Hints:_
- You need multiple joins.
- Only select the grade average (and the id for pandas `index_col`).

In [ ]:
# Add your code here:



In [ ]:
# Test with this cell:
avg_grade_result = compute_average_grade('Luanna', 'Guzzo')
for student_id, avg_grade in avg_grade_result.itertuples():
    print(f"Average Grade: {avg_grade}")  # should be 3.730...... (like in the task above)

In [ ]:
# Test Case
from unittest import TestCase
test_case = TestCase()

# Sanity Check
test_case.assertTrue('average_grade' in locals(), msg='You have to call it `average_grade`.')

# Actual Check
test_case.assertEqual(3.19, compute_average_grade('Lindsay', 'Barreneche').round(2).iloc[0].item())
test_case.assertEqual(3.85, compute_average_grade('Anisha', 'Strittmatter').round(2).iloc[0].item())
test_case.assertEqual(3.95, compute_average_grade('Gordon', 'Ketchersid').round(2).iloc[0].item())
test_case.assertEqual(4.39, compute_average_grade('Danna', 'Wedner').round(2).iloc[0].item())
test_case.assertEqual(3.32, compute_average_grade('Lino', 'Mirabelli').round(2).iloc[0].item())


print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

In [ ]:
pd.read_sql(
        """
        SELECT
            *
        FROM 
            students
        """,
        DB).sample(1)

---
# Task 6: Generating A _Report Card_ (Advanced)
> **<p style="background:rgba(255, 232, 47, 0.4)">This is an advanced task. It is not required for the next quiz.<p>**

Now, we want to define a function `generate_report_card(first_name, last_name)` that generates a report card (="Zeugnis") of the given student.

This report card should be a string:
```
        REPORT CARD for {last name}, {first name} ({Date of Birth})

        {for each semester}
        {Semester} {Grade} {Course}
        {endfor}

        {if registered}
        Current Grade: {Average Grade}
        {else}
        Final Grade: {Average Grade}
        {endif}
```

    
Example for Cristobal Recar:
```

        REPORT CARD for Recar, Cristobal (2000-12-16)
    
        Taken Courses:
        
        2019HS	4.0   Beginner Micro-Biology of Implants
        2020FS	4.5   Agricultural Renaissance on Earth
        2020FS	4.75  The Mass-Duality Scheme
        2020HS	4.0   A Complex Adaptive Systems perspective on Swarm Intelligence
        2021FS	4.0   Implications of Einstein-Rosen-Bridges

        
        Current Grade: 4.25
      
    
```

In a report card, we only want the succeeded courses, so any failed course (a grade smaller than 4.0) should **not** be added to the report and also **not** be part of the average (different than Task 5). If a student took a course multiple times, it will also be printed in the report card. (As an more advanced task, you could try to select the highest grade.)

The courses report should be ordered in ascending order by the semester this course is taken. If there are multiple courses taken that semester, order the course names alphabetically in ascending order.

For this task, you do not "need" `pd.read_sql()` but rather use `DB.execute()` to receive the plain results as tuples or lists instead of DataFrames.


For example:
```python
DB.execute('SELECT first_name, last_name FROM students').fetchone()
```
will return a **Tuple** with the first result of this query. The order in the tuple will be the same as in the `SELECT`.
```python
DB.execute('SELECT first_name, last_name FROM students').fetchall()
```
The `.fetchall()` method will return a **list** of tuples with _ALL_ students from the database.

For parametrized queries, go to the documentation: [sqlite3](https://docs.python.org/3/library/sqlite3.html).


For this task, you can assume that there are no two students with the same first and last name.

In [ ]:
# Add your code here:



In [ ]:
# No test case (use the output in the task description to verify)

# Use your newly added student (yourself!) to also print a report card:

#print(generate_report_card("YOUR FIRST NAME", "YOUR LAST NAME"))
print(generate_report_card('Kara', 'Danvers'))
print(generate_report_card('Luanna', 'Guzzo'))
print(generate_report_card('Margo Norine Sabine', 'Wagganer'))
print(generate_report_card('Cristobal', 'Recar'))
print(generate_report_card('Barbera', 'Alar'))